In [ ]:
#importing libraries
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

In [ ]:
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics

In [ ]:
columns = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","m_failed_logins",
"logged_in", "num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells","num_access_files",
"num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
"same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
"dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate",
"dst_host_srv_rerror_rate","label"]

In [ ]:
attacks_type = {
'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
    }

In [ ]:
df = pd.read_csv('/content/kddcup.data_10_percent_corrected.csv',names=columns)

In [ ]:
df['label'] = df.label.apply(lambda r: attacks_type[r[:-1]])

In [ ]:
df['label'].unique()

In [ ]:
def label_reviews(row):
    attacks_type = row['label']
    if(attacks_type == 'normal'):
        return 0
    else:
        return 1

In [ ]:
def createLabelsFromReviewPoints(df):
  df['Outcome']= df.apply (lambda row: label_reviews(row), axis=1)
  return df

In [ ]:
df = createLabelsFromReviewPoints(df)

In [ ]:
df['Outcome'].unique()

In [ ]:
df.drop(['label'],axis = 1,inplace = True)

In [ ]:
df.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
len(df.columns)

42

In [ ]:
X = df.iloc[:,1:41]
Y = df.iloc[:,-1]

In [ ]:
from sklearn import preprocessing

In [ ]:
for c in X.columns:
     if str(X[c].dtype) == 'object':
            print(c, "::", X[c].dtype, X[c].value_counts())
            #print(c, "::", T[c].dtype, T[c].value_counts())
            print("=======")

le_X = preprocessing.LabelEncoder()
le_y = preprocessing.LabelEncoder()

for c in X.columns:
    if str(X[c].dtype) == 'object': 
        X[c] = le_X.fit_transform(X[c])
    #if str(T[c].dtype) == 'object':
        #T[c] = le_X.fit_transform(T[c])

Y = le_y.fit_transform(Y)
#C = le_y.fit_transform(C)

In [ ]:
X.head()

In [ ]:
scaler = Normalizer().fit(X)
X = scaler.transform(X)
np.set_printoptions(precision=3)



In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train1,X_test1,Y_train1,Y_test1 = train_test_split(X,Y, test_size = 0.3 , random_state = 101)

In [ ]:
# reshape input to be [samples, time steps, features]
X_train1 = X_train1.reshape(-1, 1, 40)
X_test1  = X_test1.reshape(-1, 1, 40)

In [ ]:
X_train1.shape

(345814, 1, 40)

In [ ]:
X_test1.shape

(148207, 1, 40)

In [ ]:
from keras.regularizers import l1,l2
from keras.layers import Bidirectional

In [ ]:
batch_size = 70

# 1. define the network
model = Sequential()
model.add(GRU(70,return_sequences=True, kernel_regularizer='l2', input_shape =(1,40)))  
model.add(Dropout(0.1))
model.add(GRU(70,return_sequences=True, kernel_regularizer='l2',))  
model.add(Dropout(0.1))
model.add(GRU(128, return_sequences=False, kernel_regularizer='l2')) 
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
from keras import backend as K
import keras
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm2layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis1.csv',separator=',', append=False)
history = model.fit(X_train1, Y_train1, batch_size=batch_size, epochs=50, validation_data=(X_test1, Y_test1),callbacks=[checkpointer,es])

In [ ]:
loss, accuracy = model.evaluate(X_test1, Y_test1)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred1 = model.predict_classes(X_test1)

In [ ]:
accuracy = accuracy_score(Y_test1, y_pred1)
recall = recall_score(Y_test1, y_pred1 , average="binary")
precision = precision_score(Y_test1, y_pred1 , average="binary")
f1 = f1_score(Y_test1, y_pred1, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.6f" %accuracy)
print("recall")
print("%.6f" %recall)
print("precision")
print("%.6f" %precision)
print("f1score")
print("%.6f" %f1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

loss_train = np.array(history.history['loss'])
loss_val = np.array(history.history['val_loss'])

epochs = range(0,50)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(0,50)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(Y_test1,y_pred1)
plot_confusion_matrix(conf_mat=mat)